In [1]:
import numpy as np

# Utility functions

In [4]:
def softmax(x):
  e_x = np.exp(x - np.max(x))
  
  return e_x / e_x.sum(axis=0)

In [7]:
def rnn_cell_forward(timestep_t, a_prev, parameters):
  """
  Implements a single forward step of an RNN cell

  Args:
    timestep_t: (numpy array) input data at timestep "t"
    a_prev: (numpy array) hidden state at timestep "t-1"
    parameters: (dict) containing:
      Wax: (numpy array) weight matrix multiplying the input
      Waa: (numpy array) weight matrix multiplying the hidden state
      Wya: (numpy array) weight matrix relating the hiden state
        to the output
      ba: (numpy array) bias
      by: (numpy array) bias relating the hidden state to the output

  Returns:
    a_next: (numpy array) next hidden state
    yt_pred: (numpy array) prediction at timestep "t"
    cache: (tuple) with values needed for the backward pass
      it contains (a_nex, a_prev, timestep_t, parameters)
  """

  Wax = parameters['Wax']
  Waa = parameters['Waa']
  Wya = parameters['Wya']
  ba = parameters['ba']
  by = parameters['by']

  # Compute next activation state
  a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, timestep_t) + ba)

  # Compute output of the current cell
  yt_pred = softmax(np.dot(Wya, a_next) + by)

  cache = (a_next, a_prev, timestep_t, parameters)

  return a_next, yt_pred, cache

# RNN forward pass

In [11]:
def rnn_forward(x, act0, parameters):
  """
  Implements the forward propagation of a recurrent neural
  network

  Args:
    x: (numpy array) input data for every timestep
    act0: (numpy array) initial hidden state
    parameters: (dict) containing:
      Wax: (numpy array) weight matrix multiplying the input
      Waa: (numpy array) weight matrix multiplying the hidden state
      Wya: (numpy array) weight matrix relating the hiden state
        to the output
      ba: (numpy array) bias
      by: (numpy array) bias relating the hidden state to the output

  Returns:
    activations: (numpy array) hidden states for every timestep
    y_pred: (numpy array) predictions for every timestep
    caches: (tuple) with values needed for the backward pass
      it contains the list of caches and x
  """

  caches = []

  # get dimensions from shapes of x and Wy
  n_x, m, T_x = x.shape
  n_y, n_a = parameters['Wya'].shape

  # initialize 'a' and 'y' with zeros
  activations = np.zeros((n_a, m, T_x))
  y_pred = np.zeros((n_y, m, T_x))

  # initialize a_next
  a_next = act0

  for timestep in range(T_x):
    # update next hidden state, compute the prediction
    a_next, yt_pred, cache = rnn_cell_forward(x[:, :, timestep], a_next, parameters)

    # save the value of the new 'next' hidden state and the prediction
    activations[:, :, timestep] = a_next
    y_pred[:, :, timestep] = yt_pred
    caches.append(cache)

  caches = (caches, x)

  return activations, y_pred, caches